In [ ]:
import robotic as ry
import time
import numpy as np
import os
import time
import matplotlib.pyplot as plt
C = ry.Config()
C.addFile("rai-robotModels/scenarios/pandasFight.g")

C.setJointState(np.array([ 0.  , -1.  ,  0.  , -2.  ,  0.  ,  2.  ,  0.  ,  0.  ,
       -1.  ,  0.  , -2.  ,  0.  ,  2.  ,  0.]))
q0 = C.getJointState()
C.view()

In [ ]:
komo = ry.KOMO(C, 10, 10, 2, True)
komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])
komo.addObjective([5,10], ry.FS.positionDiff, ['r_gripper', "shi-h1"], ry.OT.eq, [1e1])
komo.addObjective([5.,10.], ry.FS.scalarProductZZ, ['r_gripper', 'shi-h1'], ry.OT.eq, [1e1], [])
komo.addObjective([5.,10.], ry.FS.scalarProductXZ, ['shi-h1','r_gripper'], ry.OT.eq, [1e1], [])
komo.addObjective([5.,10.], ry.FS.scalarProductXZ, ['r_gripper','shi-h1'], ry.OT.eq, [1e1], [])

# komo.addObjective([3,5], ry.FS.position,['l_gripper'], ry.OT.eq, [1e1], grasp_pos + np.array([0, 0, 0.5]) )

# komo.addObjective([5,10], ry.FS.position,['l_gripper'], ry.OT.eq, [1e1], grasp_pos )
# komo.addObjective([5,10], ry.FS.vectorX, ['l_gripper'], ry.OT.eq, [1e1],normal)

ret = ry.NLP_Solver() \
.setProblem(komo.nlp()) \
.setOptions( stopTolerance=1e-2, verbose=4 ) \
.solve()

take_shield = komo.getPath()


komo = ry.KOMO(C, 10, 10, 2, True)
komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])
komo.addObjective([5,10], ry.FS.positionDiff, ['l_gripper', "sword_0"], ry.OT.eq, [1e1])
komo.addObjective([5.,10.], ry.FS.scalarProductYZ, ['sword_0','l_gripper'], ry.OT.eq, [1e1], [])
komo.addObjective([5.,10.], ry.FS.scalarProductXZ, ['sword_0','l_gripper'], ry.OT.eq, [1e1], [])
komo.addObjective([5.,10.], ry.FS.scalarProductXX, ['l_gripper','sword_0'], ry.OT.eq, [1e1], [])

# komo.addObjective([3,5], ry.FS.position,['l_gripper'], ry.OT.eq, [1e1], grasp_pos + np.array([0, 0, 0.5]) )

# komo.addObjective([5,10], ry.FS.position,['l_gripper'], ry.OT.eq, [1e1], grasp_pos )
# komo.addObjective([5,10], ry.FS.vectorX, ['l_gripper'], ry.OT.eq, [1e1],normal)

ret = ry.NLP_Solver() \
.setProblem(komo.nlp()) \
.setOptions( stopTolerance=1e-2, verbose=4 ) \
.solve()

take_sword= komo.getPath()
armed=np.concatenate([take_sword[-1,0:7], take_shield[-1,7:]])
S = ry.Simulation(C, ry.SimulationEngine.physx, verbose=0)

In [4]:
def simulationCloseGrippers(S, tau=0.01):
    S.closeGripper('l_gripper', width=0.0001, speed=0.5)
    while (not S.getGripperIsGrasping('l_gripper')) and (S.getGripperWidth('l_gripper') > 0.001):
        time.sleep(tau)
        S.step([], tau, ry.ControlMode.none)
        #C.view()
    
    S.closeGripper('r_gripper', width=0.0001, speed=0.5)
    while (not S.getGripperIsGrasping('r_gripper')) and (S.getGripperWidth('r_gripper') > 0.001):
        time.sleep(tau)
        S.step([], tau, ry.ControlMode.none)
        #C.view()


def simulationOpenGrippers(S, tau=0.01):
    S.openGripper('l_gripper', width=0.05, speed=0.5)
    while S.getGripperWidth('l_gripper') < 0.05 - 0.01:
        time.sleep(tau)
        S.step([], tau, ry.ControlMode.none)
        #C.view()
    S.openGripper('r_gripper', width=0.05, speed=0.5)
    while S.getGripperWidth('r_gripper') < 0.05 - 0.01:
        time.sleep(tau)
        S.step([], tau, ry.ControlMode.none)
        #C.view()

def simulationGoTo(S, q, tau=0.01, checkCol=False):
    checkColTime = 0.1
    timer = 0
    while np.linalg.norm(S.get_q() - q) > 0.01:
        time.sleep(tau)
        S.step(q, tau, ry.ControlMode.position)
        timer += tau
        #C.view()
        if timer > 4:
            print("Target cannot be reached within 4 seconds.")
            break
        if checkCol:
            if timer > checkColTime:
                if findCollision(C, 'sword_1'):
                    return
                else:
                    checkColTime += 0.1

def simulationGoHome(S,tau=0.01):
    checkColTime = 0.1
    timer = 0
    while np.linalg.norm(S.get_q() - q0) > 0.01:
        time.sleep(tau)
        S.step(q0, tau, ry.ControlMode.position)
        #C.view()
        timer += tau
        # if timer > checkColTime:
        #     if findCollision(C, 'sword_1'):
        #         break
        #     else:
        #         checkColTime += 0.1
            
def simulationFollowPath(S, path, tau=0.01):
    checkColTime = 0.1
    timer = 0
    for i in range(len(path)):
        while np.linalg.norm(S.get_q() - path[i]) > 0.3:
            time.sleep(tau)
            print(i)
            print(np.linalg.norm(S.get_q() - path[i]))
            S.step(path[i], tau, ry.ControlMode.position)
            #C.view()
            timer += tau
            if timer > checkColTime:
                print('check_col')
                if findCollision(C, 'sword_1'):
                    return
                else:
                    checkColTime += 0.1

    simulationGoTo(S,C, path[-1], checkCol=True)

def simulationWait(S, t, tau=0.01):
    for k in range(int(t / tau)):
        time.sleep(tau)
        S.step([], tau, ry.ControlMode.none)
        #C.view()

def simulationTorqueCtrl(S, t, torque, tau=0.01):
    for k in range(int(t / tau)):
        time.sleep(tau)
        S.step(torque, tau, ry.ControlMode.acceleration)
        #C.view()

def simulationVelocityCtrl(S,t,vel,tau=0.01):
    for k in range(int(t/tau)):
        time.sleep(tau)
        S.step(vel,tau,ry.ControlMode.velocity)
        #C.view()

def followInterpolatedPath(S,path,tau=0.01):
    checkColTime = 0.1
    timer = 0
    for i in range(len(path)):
        time.sleep(tau)
        print(i)
        print(np.linalg.norm(S.get_q()- path[i]))
        S.step(path[i],tau,ry.ControlMode.position)
        #C.view()
        timer += tau
        if timer > checkColTime:
            if findCollision(C,'sword_1'):
                return
            else:
                checkColTime += 0.1
    simulationGoTo(S,path[-1],checkCol=True)

def findCollision(C,object1):
    collisions = [col for col in C.getCollisions(0) if object1 in col and not col[1].startswith('l_') and not col[0].startswith('l_')]
    if len(collisions) > 0:
        return True
    else:
        return False
    
def findCollisionshield(C):
    collisions = [col for col in C.getCollisions(0) if 'shi' in col and 'sword_1' in col]
    if len(collisions) > 0:
        return True
    else:
        return False
    
def findCollisionsword(C):
    collisions = [col for col in C.getCollisions(0) if 'sword_1' in col ]
    if len(collisions) > 0:
        return True
    else:
        return False
# Use this code. It is awesome. It will follow the path and stop if there is a collision.    
def followSplinePath(S,path,t,tau=0.01):
    """
        Guides a simulation object along a specified spline path.
        Parameters:
        S (Simulation): The simulation object that will follow the path.
        path (array): A list of waypoints defining the spline path.
        t (float): The total time duration for following the path.
        tau (float, optional): The time step for each simulation step. Default is 0.01.
        The function performs the following steps:
        1. Resets the spline reference of the simulation object.
        2. Sets the spline reference with the given path and a time vector.
        3. Iteratively steps through the simulation, updating the view and checking for collisions.
        4. If a collision with 'sword_1' is detected, the function exits early.
        5. The loop runs for a duration slightly longer than `t` to ensure completion.
        Note:
        - The `+20` in the loop range is a heuristic to prevent early termination when `t` is low.
        - The function checks for collisions every 0.1 seconds.

    """
    checkColTime = 0.1
    timer = 0
    S.resetSplineRef() # Reset previous spline reference
    S.setSplineRef(path,np.linspace(0.01,t,len(path))) # Set new spline reference
    joint_data = np.empty((int(t/tau)+20, 2,S.get_q().shape[0])) # Initialize array to store joint data
    for k in range(int(t/tau)+20): # This +20 is heuristic. It was stopping after a short time when t was low.
        time.sleep(tau)
        S.step([],tau,ry.ControlMode.spline)
        inst_pos = S.get_q().copy()
        inst_vel = S.get_qDot().copy()
        C.view()
        timer += tau
        joint_data[k,0,:] = inst_pos
        joint_data[k,1,:] = inst_vel
        if timer > checkColTime: # Check for collisions every 0.1 seconds
            if findCollision(C,'sword_1'): # Check if sword is colliding with something (left robot is excluded (no self collision))
                return joint_data[:k,:,:] # Return the joint data until collision
            else:
                checkColTime += 0.1 # Increment the collision check time

    return joint_data
    #simulationGoTo(S,path[-1],checkCol=True) 
def initializeSimulation(init_pos):

    S = ry.Simulation(C, ry.SimulationEngine.physx, verbose=0)

    simulationGoTo(S,armed)
    simulationCloseGrippers(S)
    simulationGoTo(S,init_pos)

    shield = C.getFrame('shi')
    robot_base = C.getFrame('r_panda_base')
    sword = C.getFrame('sword_1')
    C.view()
    return S,shield,robot_base,sword

In [11]:
attack = np.load("real_spline_attackPaths/joint_data_0.npy")
defence = np.load("defence paths/joint_data_0.npy")
print(attack.shape)
print(defence.shape)

(49, 2, 14)
(48, 2, 14)


In [15]:
import os 
import time
tau = 0.01


for file in os.listdir('real_spline_attackPaths'):

    C = ry.Config()
    C.addFile("rai-robotModels/scenarios/pandasFight.g")
    q0 = C.getJointState()
    C.setJointState(np.array([ 0.  , -1.  ,  0.  , -2.  ,  0.  ,  2.  ,  0.  ,  0.  ,
    -1.  ,  0.  , -2.  ,  0.  ,  2.  ,  0.]))

    fight_path = np.load("defence paths/joint_data_0.npy")
    init_pos = fight_path[0,0,:]
    time.sleep(1)
    S,shield,robot_base,sword = initializeSimulation(init_pos)
    k = 0
    fight_len = fight_path.shape[0]
    k = 0
    for row in fight_path:
        S.step(row[0,:], tau, ry.ControlMode.position)
        vel = S.get_qDot()
        print("attack")
        print(np.mean(np.abs(vel[:7])))
        print("defence")
        print(np.mean(np.abs(vel[7:])))
        C.view()
        time.sleep(0.1)
        k+=1
    print(f"{k/fight_len*100}% is done")
    del S
    del C

        

-- kin_physx.cpp:addJoint:298(0) ADDING JOINT l_panda_joint7-sword_0 of type rigid with rel [0, 0, 0]
-- kin_physx.cpp:addJoint:298(0) ADDING JOINT r_panda_joint7-shi of type rigid with rel [0, 0, 0]
attack
0.011759961439695741
defence
0.01635983763428937
attack
0.33892353384622503
defence
0.8831597076995032
attack
0.6581041585387928
defence
1.3349495423691613
attack
0.9223904311656952
defence
1.9021362428154265
attack
1.1315429380961828
defence
2.416209020784923
attack
1.2922736685723066
defence
2.8971748437200273
attack
1.411870362503188
defence
2.975925011294229
attack
1.514938103833369
defence
3.454516359737941
attack
1.5857317599334888
defence
3.851021421807153
attack
1.5738551205556308
defence
4.3723954473223
attack
1.5643204519791263
defence
4.614820872034345
attack
1.623844709779535
defence
4.199994044644492
attack
1.648537156837327
defence
3.7986717394420078
attack
1.6663522209439958
defence
3.4650952773434773
attack
1.6700517077531134
defence
3.3211378157138824
attack
1.66352

KeyboardInterrupt: 